In [24]:
from google.colab import files, drive
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import zipfile
from tensorflow import keras
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv
from keras.datasets import mnist
from sklearn.model_selection import train_test_split


In [25]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Kaggle API

In [26]:
# #you need to create your own API first in kaggle
# #upload kaggle json

# files.upload()

In [27]:
# #create a kaggle folder
# !mkdir ~/.kaggle

In [28]:
# #copy kaggle.json to folder created
# !cp kaggle.json ~/.kaggle/

# #permission for the json to act
# ! chmod 600 ~/.kaggle/kaggle.json

In [29]:
# # A-Z Dataset
# !kaggle kernels pull code/naaggss/starter-handwritten-a-z-66906975-6

# Load A-Z Dataset

In [ ]:
test_local_zip = '/content/drive/MyDrive/ML Capstone Bangkit/Dataset/handwritten_data_785.csv.zip'
zip_ref = zipfile.ZipFile(test_local_zip, 'r')
zip_ref.extractall('/tmp/dataset')

zip_ref.close()

In [ ]:
os.listdir('/tmp/dataset/')

In [ ]:
alphabet_data = '/tmp/dataset/handwritten_data_785.csv'
aZ_data = []
aZ_labels = []

files = open(alphabet_data)

for row in files:
  row  = row.split(',')

  #append labels. Row 0 = labels / header
  aZ_labels.append(int(row[0]))

  #append labels. Row 1:
  image = np.array([int(x) for x in row[1:]], dtype="uint8")
  img = image.reshape((28,28))
  aZ_data.append(img)

aZ_labels = np.array(aZ_labels, dtype = 'int')
aZ_data = np.array(aZ_data, dtype='float32')

In [ ]:
print(len(aZ_labels))
print(len(aZ_data))

In [ ]:
print(aZ_labels[-1])
print(aZ_data[-1].shape)

# Load MNIST Digit Dataset

In [ ]:
(train_digit_data, train_digit_labels), (test_digit_data, test_digit_labels) = mnist.load_data()

digit_data = np.vstack([train_digit_data, test_digit_data])
digit_labels = np.hstack([train_digit_labels, test_digit_labels])

In [ ]:
# from matplotlib import pyplot
# for i in range(9):  
#   pyplot.subplot(330 + 1 + i)
#   pyplot.title('label: {}'.format(digit_labels[i]))
#   pyplot.imshow(digit_data[i], cmap=pyplot.get_cmap('gray'))
#   pyplot.show()

In [ ]:
# the MNIST dataset occupies the labels 0-9, so let's add 10 to every A-Z label to ensure the A-Z characters are not incorrectly labeled 

aZ_labels += 10

# stack the A-Z data and labels with the MNIST digits data and labels

data = np.vstack([aZ_data, digit_data])
labels = np.hstack([aZ_labels, digit_labels])

# Each image in the A-Z and MNIST digts datasets are 28x28 pixels;
# However, the architecture we're using is designed for 32x32 images,
# So we need to resize them to 32x32

data = [cv2.resize(image, (32, 32)) for image in data]
data = np.array(data, dtype="float32")

# add a channel dimension to every image in the dataset and scale the
# pixel intensities of the images from [0, 255] down to [0, 1]

data = np.expand_dims(data, axis=-1)
data /= 255.0

In [ ]:
print(len(aZ_labels))

In [ ]:
print("amount of data: {}".format(len(data)))
print("amount of labels: {}".format(len(labels)))
print("amount of class: {}".format(len(np.unique(labels))))

In [ ]:
#split data
# Split Dataset
x_train, x_test, y_train, y_test = train_test_split(data, labels, train_size = 0.8)

x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
print("amount of train data: {}".format(len(x_train)))
print("amount of test data: {}".format(len(x_test)))
print("shape of data: {}".format(x_train.shape[1:4]))

In [ ]:
img_size = 32
EPOCHS = 100
INIT_LR = 1e-1
batch_size = 128

In [ ]:
# construct the image generator for data augmentation
train_aug = ImageDataGenerator(
  rescale = 1./255,
  rotation_range=10,
  zoom_range=0.05,
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.15,
  horizontal_flip=False,
  fill_mode="nearest")

valid_aug = ImageDataGenerator(
    rescale = 1./255
)
train_datagen = train_aug.flow(x_train, y_train, batch_size = batch_size)

validation_datagen = valid_aug.flow(x_test, y_test, batch_size = batch_size)

In [ ]:
model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32,(3,3), activation ='relu', input_shape=(32,32,1)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(64,(3,3), activation ='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128,(3,3), activation ='relu'),
      tf.keras.layers.MaxPooling2D(2,2),

      tf.keras.layers.Flatten(),
      tf.keras.layers.Dropout(0.5),
 
      tf.keras.layers.Dense(512, activation = 'relu'),
      tf.keras.layers.Dense(36, activation='softmax'),

])
model.build(input_shape = (None, 32, 32, 1))
model.summary()

In [ ]:
# model = ResNet.build(32, 32, 1, len(le.classes_), (3, 3, 3),
# # (64, 64, 128, 256), reg=0.0005)
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [ ]:
History = model.fit(
train_datagen,
validation_data=validation_datagen,
steps_per_epoch=len(x_train) // batch_size, 
epochs=EPOCHS,
verbose=2)

In [ ]:
!mkdir -p ocr_model
model.save('ocr-model-Felix-4.h5')

In [ ]:
model = load_model('ocr-model-Felix-4.h5')